In [10]:
import pandas as pd
import re

tweets_df = pd.read_csv('tweetlist.csv')

# Duplicated satırları sil
tweets_df = tweets_df.drop_duplicates()


tweets_filtered = tweets_df.copy() #it's a good idea to work on the copy of original dataframe, so we can always go back to it if we mess something up
column_list = ["tweet_id","created_at", "author_id", "text","hashtag"]
tweets_filtered = tweets_filtered[column_list]
tweets_filtered



,tweet_id,created_at,author_id,text,hashtag
0,1653187356525228032,2023-05-01 23:59:15+00:00,1253475685433176065,RT @KiNG_NiCO_18: Proof the LGBT is satanic #g...,"#greenscreenvideo, #greenscreen, #LGBT, #Pride..."
1,1653187307183423488,2023-05-01 23:59:03+00:00,1364784121755574274,RT @FortniteGod1089: AROMANTIC RICE😮 #Pride ht...,#Pride
2,1653187158847672320,2023-05-01 23:58:28+00:00,966744273755361280,RT @thatcubnextdoor: Some pictures with one of...,"#tummy, #jock, #jockst"
3,1653186933697417216,2023-05-01 23:57:34+00:00,1545067267603320832,I just published my website! 🎉 https://t.co/dz...,"#pride, #blog, #gettingstarted"
4,1653186559447912448,2023-05-01 23:56:05+00:00,1520099530145939461,RT @acvee_29: LESBIAN VISIBILITY WEEK EVERYONE...,"#lesbian, #AmityBlight, #toh, #pride"
...,...,...,...,...,...
4950,1666339147186049025,2023-06-07T06:59:46.000Z,829526729110597632,Only way to do it #Pride 🌈 https://t.co/LioJnU...,#Pride
4951,1666338998674149376,2023-06-07T06:59:11.000Z,299496213,"Celebrating pride month by wearing Green, as i...","#Pride, #pridemonth, #HappyPrideMonth, #pride2..."
4952,1666338857472720897,2023-06-07T06:58:37.000Z,1382251498281828353,"Sapphic pockets of bookish joy, for @booksthat...",#pride
4953,1666338473425371136,2023-06-07T06:57:05.000Z,1561737360739631108,Happy #pride to alllllllllllll the freaky slut...,#pride


In [19]:
import re
import string
# NLTK tools
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words("english")
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
from collections import defaultdict
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [20]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Emoji'leri tanımlamak için bir regex deseni
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # simgeler ve pictographs
                           u"\U0001F680-\U0001F6FF"  # taşıtlar
                           u"\U0001F1E0-\U0001F1FF"  # bayraklar
                           u"\U00002500-\U00002BEF"  # Çin, Japon, Kore ve diğer ideografik karakterler
                           u"\U00002702-\U000027B0"  # Dingbats
                           u"\U00002702-\U000027B0"  # Orta Doğu Süreç Simgeleri
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

In [21]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # mentions kaldırılıyor
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # hashtag'ler kaldırılıyor
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # http bağlantıları kaldırılıyor
    tweet = " ".join(tweet.split())
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if w.lower() in words and not w.lower() in stop_words)
    # stop words'ler kaldırılıyor
    lemma_function = WordNetLemmatizer()
    tweet = " ".join(lemma_function.lemmatize(token, tag_map[tag[0]]) for token, tag in nltk.pos_tag(nltk.wordpunct_tokenize(tweet))) # lemmatize
    tweet = str.lower(tweet) # küçük harflere dönüştürülüyor
    tweet = emoji_pattern.sub(r"", tweet) # emoji'ler kaldırılıyor
    return tweet

def cleantext(tweet):
  tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet)
  tweet = emoji_pattern.sub(r"", tweet) # emoji'ler kaldırılıyor
  return tweet

In [35]:
# Veri çerçevesini oluştur
df = pd.DataFrame(tweets_filtered)

# RT ifadelerini temizle ve sadece orijinal metni al
df['text'] = df['text'].str.replace(r'^RT\s+@\w+:\s', '', regex=True)

# HTTP linklerini kaldır
df['text'] = df['text'].str.replace(r'http\S+', '', regex=True)

# Tekrar eden metinleri sadece birini tut
df = df.drop_duplicates(subset='text')

# Sadece text sütununu içeren yeni bir veri çerçevesi oluştur
only_text = pd.DataFrame(df['text'])

# Yeni veri çerçevesini görüntüle
only_text.to_csv("only_text")
only_text

,text
0,Proof the LGBT is satanic #greenscreenvideo #g...
1,AROMANTIC RICE😮 #Pride
2,Some pictures with one of the new jocks I got ...
3,I just published my website! 🎉 #pride #blog #...
4,LESBIAN VISIBILITY WEEK EVERYONE🗣️‼️‼️‼️💞\n.\n...
...,...
4950,Only way to do it #Pride 🌈
4951,"Celebrating pride month by wearing Green, as i..."
4952,"Sapphic pockets of bookish joy, for @booksthat..."
4953,Happy #pride to alllllllllllll the freaky slut...


In [36]:
tweets_filtered["clean_text"] = tweets_filtered["text"].map(cleaner)
tweets_filtered

,tweet_id,created_at,author_id,text,hashtag,clean_text
0,1653187356525228032,2023-05-01 23:59:15+00:00,1253475685433176065,Proof the LGBT is satanic #greenscreenvideo #g...,"#greenscreenvideo, #greenscreen, #LGBT, #Pride...",proof satanic
1,1653187307183423488,2023-05-01 23:59:03+00:00,1364784121755574274,AROMANTIC RICE😮 #Pride,#Pride,rice
2,1653187158847672320,2023-05-01 23:58:28+00:00,966744273755361280,Some pictures with one of the new jocks I got ...,"#tummy, #jock, #jockst",one new get yesterday four take
3,1653186933697417216,2023-05-01 23:57:34+00:00,1545067267603320832,I just published my website! 🎉 #pride #blog #...,"#pride, #blog, #gettingstarted",
4,1653186559447912448,2023-05-01 23:56:05+00:00,1520099530145939461,LESBIAN VISIBILITY WEEK EVERYONE🗣️‼️‼️‼️💞\n.\n...,"#lesbian, #AmityBlight, #toh, #pride",visibility week everyone draw beloved favorite
...,...,...,...,...,...,...
4950,1666339147186049025,2023-06-07T06:59:46.000Z,829526729110597632,Only way to do it #Pride 🌈,#Pride,way
4951,1666338998674149376,2023-06-07T06:59:11.000Z,299496213,"Celebrating pride month by wearing Green, as i...","#Pride, #pridemonth, #HappyPrideMonth, #pride2...",pride month wear green nature growth harmony
4952,1666338857472720897,2023-06-07T06:58:37.000Z,1382251498281828353,"Sapphic pockets of bookish joy, for @booksthat...",#pride,sapphic bookish joy june box
4953,1666338473425371136,2023-06-07T06:57:05.000Z,1561737360739631108,Happy #pride to alllllllllllll the freaky slut...,#pride,happy freaky


In [37]:
tweets_filtered.loc[tweets_filtered["clean_text"].isnull(),"clean_text"] = ""
tweets_filtered.to_csv("tf.csv")

In [38]:
tweet_tokenizer = nltk.TweetTokenizer()

#initialize an empty dict
unique_words = {}

for idx, row in tweets_filtered.iterrows():
  if row["clean_text"] != "":
    for word in tweet_tokenizer.tokenize(row["clean_text"]):
      unique_words.setdefault(word,0)
      unique_words[word] += 1

In [39]:
uw_df = pd.DataFrame.from_dict(unique_words, orient='index').reset_index()
uw_df.rename(columns = {'index':'Label', 0:'Count'}, inplace=True)
uw_df.sort_values(by=['Count'], ascending=False, inplace=True)
uw_df = uw_df.reset_index().drop(columns=["index"])
uw_df

,Label,Count
0,pride,1009
1,happy,692
2,month,540
3,love,325
4,get,265
...,...,...
3713,contentious,1
3714,federalist,1
3715,enforced,1
3716,exercise,1


In [40]:
uw_df.to_csv("wordsnodes.csv")

In [41]:
tweets_filtered["hashtags"] = ""


In [42]:
unique_hashtags = {}

for idx, row in tweets_filtered.iterrows():
    text = row["text"]
    hashtags = []
    for word in text.split():
        if word.startswith("#"):
            word = word[1:]
            hashtag = word.lower()
            unique_hashtags.setdefault(hashtag, 0)
            unique_hashtags[hashtag] += 1
            hashtags.append(hashtag)

    tweets_filtered.at[idx, "hashtags"] = hashtags


In [43]:
unique_hashtags = dict(sorted(unique_hashtags.items(), key=lambda item: item[1], reverse=True))

In [44]:
uh_df = pd.DataFrame.from_dict(unique_hashtags, orient='index').reset_index()
uh_df.rename(columns = {'index':'Label', 0:'Count'}, inplace=True)
uh_df.to_csv('hashtagnodes.csv')

In [45]:
import itertools
import networkx as nx

In [46]:
uh = unique_hashtags.keys()
uw = unique_words.keys()


In [47]:
network = {}
network_key = 0
for index, row in tweets_filtered.iterrows():
    combined_list = [hashtag for hashtag in row["hashtags"]] + [word for word in str.split(row["clean_text"], " ") ]
    #if word in nodes["Label"].tolist()
    #itertool product creates Cartesian product of each element in the combined list
    for pair in itertools.product(combined_list, combined_list):
        #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
        if pair[0]!=pair[1] and not(pair[::-1] in network):
            network.setdefault(pair,0)
            network[pair] += 1

network_df = pd.DataFrame.from_dict(network, orient="index")

In [48]:
network_df.reset_index(inplace=True)
network_df.columns = ["pair","weight"]
network_df.sort_values(by="weight",inplace=True, ascending=False)
network_df = network_df[network_df['weight']>1]
network_df

,pair,weight
960,"(pride, month)",829
285,"(pride, happy)",797
809,"(pridemonth, pride)",685
171,"(pride, gay)",460
251,"(pride, love)",455
...,...,...
69903,"(boycott, nation)",2
95873,"(month, video)",2
17904,"(translivesmatter, bill)",2
120344,"(everybody, community)",2


In [50]:
new_df = network_df.copy()
new_df[['source', 'target']] = new_df['pair'].astype(str).str.strip('()').str.split(', ', expand=True)
new_df = new_df[['source', 'target', 'weight']]
new_df['source'] = new_df['source'].str.strip("'")
new_df['target'] = new_df['target'].str.strip("'")
new_df
#new_df=new_df.to_csv("edges.csv", sep=';')
new_df.to_csv("alledges.csv")

<ipython-input-50-d4fa9facbbbc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['source'] = new_df['source'].str.strip("'")
<ipython-input-50-d4fa9facbbbc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['target'] = new_df['target'].str.strip("'")
